Lambda School Data Science

*Unit 2, Sprint 1, Module 3*

---

# Ridge Regression

## Assignment

We're going back to our other **New York City** real estate dataset. Instead of predicting apartment rents, you'll predict property sales prices.

But not just for condos in Tribeca...

- [ ] Use a subset of the data where `BUILDING_CLASS_CATEGORY` == `'01 ONE FAMILY DWELLINGS'` and the sale price was more than 100 thousand and less than 2 million.
- [ ] Do train/test split. Use data from January — March 2019 to train. Use data from April 2019 to test.
- [ ] Do one-hot encoding of categorical features.
- [ ] Do feature selection with `SelectKBest`.
- [ ] Fit a ridge regression model with multiple features. Use the `normalize=True` parameter (or do [feature scaling](https://scikit-learn.org/stable/modules/preprocessing.html) beforehand — use the scaler's `fit_transform` method with the train set, and the scaler's `transform` method with the test set)
- [ ] Get mean absolute error for the test set.
- [ ] As always, commit your notebook to your fork of the GitHub repo.

The [NYC Department of Finance](https://www1.nyc.gov/site/finance/taxes/property-rolling-sales-data.page) has a glossary of property sales terms and NYC Building Class Code Descriptions. The data comes from the [NYC OpenData](https://data.cityofnewyork.us/browse?q=NYC%20calendar%20sales) portal.


## Stretch Goals

Don't worry, you aren't expected to do all these stretch goals! These are just ideas to consider and choose from.

- [ ] Add your own stretch goal(s) !
- [ ] Instead of `Ridge`, try `LinearRegression`. Depending on how many features you select, your errors will probably blow up! 💥
- [ ] Instead of `Ridge`, try [`RidgeCV`](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.RidgeCV.html).
- [ ] Learn more about feature selection:
    - ["Permutation importance"](https://www.kaggle.com/dansbecker/permutation-importance)
    - [scikit-learn's User Guide for Feature Selection](https://scikit-learn.org/stable/modules/feature_selection.html)
    - [mlxtend](http://rasbt.github.io/mlxtend/) library
    - scikit-learn-contrib libraries: [boruta_py](https://github.com/scikit-learn-contrib/boruta_py) & [stability-selection](https://github.com/scikit-learn-contrib/stability-selection)
    - [_Feature Engineering and Selection_](http://www.feat.engineering/) by Kuhn & Johnson.
- [ ] Try [statsmodels](https://www.statsmodels.org/stable/index.html) if you’re interested in more inferential statistical approach to linear regression and feature selection, looking at p values and 95% confidence intervals for the coefficients.
- [ ] Read [_An Introduction to Statistical Learning_](http://faculty.marshall.usc.edu/gareth-james/ISL/ISLR%20Seventh%20Printing.pdf), Chapters 1-3, for more math & theory, but in an accessible, readable way.
- [ ] Try [scikit-learn pipelines](https://scikit-learn.org/stable/modules/compose.html).

In [1]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Applied-Modeling/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'
    
# Ignore this Numpy warning when using Plotly Express:
# FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
import warnings
warnings.filterwarnings(action='ignore', category=FutureWarning, module='numpy')

In [2]:
import pandas as pd
import pandas_profiling

# Read New York City property sales data
df = pd.read_csv(DATA_PATH+'condos/NYC_Citywide_Rolling_Calendar_Sales.csv')

# Change column names: replace spaces with underscores
df.columns = [col.replace(' ', '_') for col in df]

# SALE_PRICE was read as strings.
# Remove symbols, convert to integer
df['SALE_PRICE'] = (
    df['SALE_PRICE']
    .str.replace('$','')
    .str.replace('-','')
    .str.replace(',','')
    .astype(int)
)

In [3]:
# BOROUGH is a numeric column, but arguably should be a categorical feature,
# so convert it from a number to a string
df['BOROUGH'] = df['BOROUGH'].astype(str)

In [4]:
# Reduce cardinality for NEIGHBORHOOD feature

# Get a list of the top 10 neighborhoods
top10 = df['NEIGHBORHOOD'].value_counts()[:10].index

# At locations where the neighborhood is NOT in the top 10, 
# replace the neighborhood with 'OTHER'
df.loc[~df['NEIGHBORHOOD'].isin(top10), 'NEIGHBORHOOD'] = 'OTHER'

In [17]:
# Take a look at the
df.tail()

,BOROUGH,NEIGHBORHOOD,BUILDING_CLASS_CATEGORY,TAX_CLASS_AT_PRESENT,BLOCK,LOT,EASE-MENT,BUILDING_CLASS_AT_PRESENT,ADDRESS,APARTMENT_NUMBER,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,LAND_SQUARE_FEET,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE,BUILDING_CLASS_AT_TIME_OF_SALE,SALE_PRICE,SALE_DATE
23035,4,OTHER,01 ONE FAMILY DWELLINGS,1,10965,276,NaN,A5,111-17 FRANCIS LEWIS BLVD,NaN,11429.0,1.0,0.0,1.0,"1,800",1224.0,1945.0,1,A5,510000,2019-04-30
23036,4,OTHER,09 COOPS - WALKUP APARTMENTS,2,169,29,NaN,C6,"45-14 43RD STREET, 3C",NaN,11104.0,0.0,0.0,0.0,0,0.0,1929.0,2,C6,355000,2019-04-30
23037,4,OTHER,10 COOPS - ELEVATOR APARTMENTS,2,131,4,NaN,D4,"50-05 43RD AVENUE, 3M",NaN,11377.0,0.0,0.0,0.0,0,0.0,1932.0,2,D4,375000,2019-04-30
23038,4,OTHER,02 TWO FAMILY DWELLINGS,1,8932,18,NaN,S2,91-10 JAMAICA AVE,NaN,11421.0,2.0,1.0,3.0,"2,078",2200.0,1931.0,1,S2,1100000,2019-04-30
23039,4,OTHER,12 CONDOS - WALKUP APARTMENTS,2,1216,1161,NaN,R2,"61-05 39TH AVENUE, F5",F5,11377.0,1.0,0.0,85.0,"15,151",854.0,1927.0,2,R2,569202,2019-04-30


In [136]:
# Import datetime libary
import datetime as dt

# Convert the SALE_DATE to datetime format
df['SALE_DATE'] = pd.to_datetime(df['SALE_DATE'])

In [137]:
# Create the train data
train = df[df['SALE_DATE'].dt.month <= 3]

# Create the test data
test = df[df['SALE_DATE'].dt.month == 4]

In [139]:
# One hot
# Import the calls
from sklearn.preprocessing import OneHotEncoder

# instatiate
ohe = OneHotEncoder()

# Save time to do it in one step
train_trans = ohe.fit_transform(train[['NEIGHBORHOOD']]).toarray()

# Take a look at the onehot encoder
train_trans

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [174]:
y_train = train['SALE_PRICE']
y_test = test['SALE_PRICE']
X_train = train.select_dtypes(include='number')
X_test= test.select_dtypes(include='number')

In [175]:
# Looks for NaN values
print(X_train.isnull().sum())
print(X_test.isnull().sum())

BLOCK                            0
LOT                              0
EASE-MENT                    18167
ZIP_CODE                         0
RESIDENTIAL_UNITS                0
COMMERCIAL_UNITS                 0
TOTAL_UNITS                      0
GROSS_SQUARE_FEET                0
YEAR_BUILT                       5
TAX_CLASS_AT_TIME_OF_SALE        0
SALE_PRICE                       0
dtype: int64
BLOCK                           0
LOT                             0
EASE-MENT                    4873
ZIP_CODE                        1
RESIDENTIAL_UNITS               1
COMMERCIAL_UNITS                1
TOTAL_UNITS                     1
GROSS_SQUARE_FEET               1
YEAR_BUILT                     30
TAX_CLASS_AT_TIME_OF_SALE       0
SALE_PRICE                      0
dtype: int64


In [176]:
# Drop the column EASE-MENT because all the value are Nan
X_train = X_train.drop(['EASE-MENT'], axis=1)
X_test = X_test.drop(['EASE-MENT'], axis=1)

# Replace the five NaN from YEAR_BUILT with the mean
X_train['YEAR_BUILT'].fillna(value=X_train['YEAR_BUILT'].mean(), inplace=True)
X_test['YEAR_BUILT'].fillna(value=X_test['YEAR_BUILT'].mean(), inplace=True)

In [177]:
# Looks for NaN values again
print(X_train.isnull().sum())
print(X_test.isnull().sum())

BLOCK                        0
LOT                          0
ZIP_CODE                     0
RESIDENTIAL_UNITS            0
COMMERCIAL_UNITS             0
TOTAL_UNITS                  0
GROSS_SQUARE_FEET            0
YEAR_BUILT                   0
TAX_CLASS_AT_TIME_OF_SALE    0
SALE_PRICE                   0
dtype: int64
BLOCK                        0
LOT                          0
ZIP_CODE                     1
RESIDENTIAL_UNITS            1
COMMERCIAL_UNITS             1
TOTAL_UNITS                  1
GROSS_SQUARE_FEET            1
YEAR_BUILT                   0
TAX_CLASS_AT_TIME_OF_SALE    0
SALE_PRICE                   0
dtype: int64


In [180]:
# Drop all NaN from X_test
X_test = X_test.dropna()

In [181]:
# Looks for NaN values again
print(X_train.isnull().sum())
print(X_test.isnull().sum())

BLOCK                        0
LOT                          0
ZIP_CODE                     0
RESIDENTIAL_UNITS            0
COMMERCIAL_UNITS             0
TOTAL_UNITS                  0
GROSS_SQUARE_FEET            0
YEAR_BUILT                   0
TAX_CLASS_AT_TIME_OF_SALE    0
SALE_PRICE                   0
dtype: int64
BLOCK                        0
LOT                          0
ZIP_CODE                     0
RESIDENTIAL_UNITS            0
COMMERCIAL_UNITS             0
TOTAL_UNITS                  0
GROSS_SQUARE_FEET            0
YEAR_BUILT                   0
TAX_CLASS_AT_TIME_OF_SALE    0
SALE_PRICE                   0
dtype: int64


In [186]:
# SelectKBest
# Import the feature selector utility
from sklearn.feature_selection import SelectKBest

selector = SelectKBest(k=4)
X_train_selected = selector.fit_transform(X_train, y_train)
X_test_selected = selector.transform(X_test)

# Find the features that was selected
selected_mask = selector.get_support()
all_features = X_train.columns
selected_feature = all_features[selected_mask]

print('list of the selected feature: ', selected_feature)

list of the selected feature:  Index(['COMMERCIAL_UNITS', 'TOTAL_UNITS', 'GROSS_SQUARE_FEET', 'SALE_PRICE'], dtype='object')


/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/_univariate_selection.py:115: RuntimeWarning: divide by zero encountered in true_divide
  f = msb / msw


In [187]:
# Create the target and features
target = 'SALE_PRICE'
features = selected_feature

# Create the matrix for y and X
y = train[target]
X = train[features]

# Import for sklearn the train test split 
from sklearn.model_selection import train_test_split

# Declare the variable for the slipt
X_train, X_val, y_train, y_val = train_test_split(X,y, test_size=0.25, random_state=19)

# Print the shape for X_train and y_train
print(X_train.shape)
print(y_train.shape)

(13625, 4)
(13625,)


In [189]:
# Import estimator Ridge from sklearn
from sklearn.linear_model import Ridge

# Instantiate
lin_ridge = Ridge(normalize=True, alpha=0.05)

# Fit the model
lin_ridge.fit(X_train, y_train)

# The predictions
ridge_pred = lin_ridge.predict(X_val)

# Coefficients
pd.Series(lin_ridge.coef_, features)

COMMERCIAL_UNITS     3815.013193
TOTAL_UNITS         -4915.483618
GROSS_SQUARE_FEET      17.586935
SALE_PRICE              0.936577
dtype: float64